In [1]:
import numpy as np

In [142]:
data = np.genfromtxt('./input.txt', delimiter=1).astype(np.int_)

In [144]:
data

array([[4, 8, 7, 1, 2, 5, 2, 7, 6, 3],
       [8, 5, 3, 3, 4, 2, 8, 1, 7, 3],
       [7, 1, 8, 2, 1, 8, 6, 8, 1, 3],
       [2, 1, 2, 8, 4, 4, 1, 5, 4, 1],
       [3, 7, 2, 2, 2, 7, 2, 2, 7, 2],
       [8, 7, 5, 1, 6, 8, 3, 4, 4, 3],
       [3, 1, 3, 5, 5, 7, 1, 1, 5, 3],
       [5, 8, 1, 6, 3, 2, 1, 5, 7, 2],
       [2, 6, 5, 1, 3, 4, 7, 2, 7, 1],
       [7, 7, 8, 8, 1, 5, 4, 2, 5, 2]])

In [115]:
NBR_OFFSETS = (np.indices([3, 3])-1).T.reshape(9,2)
NBR_OFFSETS = np.delete(NBR_OFFSETS, 4, axis=0)

def octo_step(octogrid):
    grid_shape = octogrid.shape[0]
    assert grid_shape == octogrid.shape[1]  # if not, handle it in the coord bounds check below
    energy_lvls = octogrid + 1
    now_flash_coords = np.argwhere(energy_lvls > 9)
    just_flashed_mask = np.zeros_like(energy_lvls)
    n_flashes = 0
    while len(now_flash_coords) > 0:
        n_flashes += len(now_flash_coords)
        for flash_pos in now_flash_coords:
            just_flashed_mask[flash_pos[0], flash_pos[1]] = 1
            flashing_nbr_coords = (NBR_OFFSETS + flash_pos)
            flashing_nbr_coords = flashing_nbr_coords[
                np.logical_and(
                    flashing_nbr_coords.min(axis=1) >= 0,  
                    flashing_nbr_coords.max(axis=1) < grid_shape)
            ].T
            energy_lvls[flashing_nbr_coords[0], flashing_nbr_coords[1]] += 1
        energy_lvls = energy_lvls * np.logical_not(just_flashed_mask)
        now_flash_coords = np.argwhere(energy_lvls > 9)
    return energy_lvls, n_flashes

In [138]:
total_flashes = 0
state = data
step_no = 0
all_flashed_step = -1

In [139]:
while(True):
    state, n_flashed = octo_step(state)
    if step_no < 100:
        total_flashes += n_flashed
    if n_flashed == 100:
        all_flashed_step = step_no + 1
        break
    step_no += 1

In [140]:
total_flashes

1747

In [141]:
all_flashed_step

505

# josh's solution to see what the answer should be...

In [36]:
# Day 11 : Dumbo Octopus

## Part 1


import numpy as np
from scipy.ndimage.morphology import binary_dilation
from scipy.ndimage import generate_binary_structure
from functools import partial


# define dilation
selem = generate_binary_structure(2, 2)
dilate = partial(binary_dilation, structure=selem)


# define class to combine flashes
class FlashMask:

    def __init__(self, shape):
        self.shape = shape
        self.array = np.zeros(shape)
        self.count = 0
    
    def __add__(self, array):
        return array + self.array

    def __new_mask(self, i, j):
        mask = np.zeros(self.shape)
        mask[i, j] = 1
        return dilate(mask)

    def add(self, i, j):
        mask = self.__new_mask(i, j)
        self.array += mask
        self.count += 1

    def reset(self):
        self.array = np.zeros(self.shape)


# load octopi
#octopi = open("input.txt").read().split("\n")
#octopi = np.array([list(o) for o in octopi], dtype=int)
octopi = np.genfromtxt('./input.txt', delimiter=1)

# initialise flash mask
flash_mask = FlashMask(octopi.shape)


# iterate days
n_days = 100
for day in range(n_days):

    # Add 1 to all
    octopi += 1

    # find flashes
    flashes = octopi > 9
    
    # while there are more to flash, keep flashing!
    while np.sum(flashes) > 0:

        # Add flashes
        for i, j in np.argwhere(flashes):
            flash_mask.add(i, j)
        octopi = flash_mask + octopi

        # update params
        octopi[flashes] = -np.inf
        flashes = octopi > 9
        flash_mask.reset()
    
    # Set flashers to 0 energy
    octopi[octopi == -np.inf] = 0


print(flash_mask.count)

1747
